## Part 5: Discrete-profiling
If multiple pdfs exist to fit some distribution, we can store all pdfs in a single workspace by using a `RooMultiPdf` object. The code blocks below show how to store the exponential, (4th order) Chebychev polynomial and the power law function from the previous section in a `RooMultiPdf` object. This requires a `RooCategory` index, which controls the pdf which is active at any one time. Look at the code and run:

In [ ]:
import ROOT
from IPython.display import Image

In [ ]:
# Define mass and weight variables
mass = ROOT.RooRealVar("CMS_hgg_mass", "CMS_hgg_mass", 125, 100, 180)
weight = ROOT.RooRealVar("weight","weight",0,0,1)

# Load the data 
f = ROOT.TFile("data_part1.root","r")
t = f.Get("data_Tag0")

data = ROOT.RooDataSet("data_Tag0", "data_Tag0", t, ROOT.RooArgSet(mass), "", "weight")

In [ ]:
# Define ranges to fit for initial parameter values
mass.setRange("loSB", 100, 115 )
mass.setRange("hiSB", 135, 180 )
mass.setRange("full", 100, 180 )
fit_range = "loSB,hiSB"

# Define the different background model pdf choices and fit to the data mass sidebands
# RooExponential
alpha = ROOT.RooRealVar("alpha", "alpha", -0.05, -0.2, 0 )
model_exp_bkg = ROOT.RooExponential("model_exp_bkg_Tag0", "model_exp_bkg_Tag0", mass, alpha )
# Fit model to data sidebands
model_exp_bkg.fitTo( data, ROOT.RooFit.Range(fit_range), ROOT.RooFit.Minimizer("Minuit2","minimize"),ROOT.RooFit.SumW2Error(True), ROOT.RooFit.PrintLevel(-1) )

# RooChebychev polynomial: 4th order
poly_1 = ROOT.RooRealVar("poly_1","T1 of chebychev polynomial", 0.01, -4, 4)
poly_2 = ROOT.RooRealVar("poly_2","T2 of chebychev polynomial", 0.01, -4, 4)
poly_3 = ROOT.RooRealVar("poly_3","T3 of chebychev polynomial", 0.01, -4, 4)
poly_4 = ROOT.RooRealVar("poly_4","T4 of chebychev polynomial", 0.01, -4, 4)
model_poly_bkg = ROOT.RooChebychev("model_poly_bkg_Tag0", "model_poly_bkg_Tag0", mass, ROOT.RooArgList(poly_1,poly_2,poly_3,poly_4) )
# Fit model to data sidebands
model_poly_bkg.fitTo( data, ROOT.RooFit.Range(fit_range), ROOT.RooFit.Minimizer("Minuit2","minimize"),ROOT.RooFit.SumW2Error(True), ROOT.RooFit.PrintLevel(-1) )

# Power law function: using RooGenericPdf functionality
pow_1 = ROOT.RooRealVar("pow_1","Exponent of power law", -3, -10, -0.0001)
model_pow_bkg = ROOT.RooGenericPdf("model_pow_bkg_Tag0", "TMath::Power(@0,@1)", ROOT.RooArgList(mass,pow_1) )
# Fit model to data sidebands
model_pow_bkg.fitTo( data, ROOT.RooFit.Range(fit_range), ROOT.RooFit.Minimizer("Minuit2","minimize"),ROOT.RooFit.SumW2Error(True), ROOT.RooFit.PrintLevel(-1) )

In [ ]:
# Make a RooCategory object: this will control which PDF is "active"
cat = ROOT.RooCategory("pdfindex_Tag0", "Index of Pdf which is active for Tag0")

# Make a RooArgList of the models
models = ROOT.RooArgList()
models.add(model_exp_bkg)
models.add(model_poly_bkg)
models.add(model_pow_bkg)

# Build the RooMultiPdf object
multipdf = ROOT.RooMultiPdf("multipdf_Tag0", "MultiPdf for Tag0", cat, models)

In [ ]:
# Define normalisation object
# As usual, data-driven fit so want the background model to have a freely floating yield
norm = ROOT.RooRealVar("multipdf_Tag0_norm", "Number of background events in Tag0", data.numEntries(), 0, 3*data.numEntries() )

In [ ]:
# Lets save the data as a RooDataHist with 320 bins between 100 and 180
mass.setBins(320)
data_hist = ROOT.RooDataHist("data_hist_Tag0", "data_hist_Tag0", mass, data )

In [ ]:
# Save the background model and data set to a RooWorkspace
f_out = ROOT.TFile("workspace_bkg_multipdf.root", "RECREATE")
w_bkg = ROOT.RooWorkspace("workspace_bkg","workspace_bkg")
getattr(w_bkg, "import")(data_hist)
getattr(w_bkg, "import")(cat)
getattr(w_bkg, "import")(norm)
getattr(w_bkg, "import")(multipdf)
w_bkg.Print()
w_bkg.Write()
f_out.Close()

The file `datacard_part5.txt` will load the multipdf as the background model. Notice the line at the end of the datacard (see below). This tells combine about the `RooCategory` index.
```
pdfindex_Tag0         discrete
```
First have a look at the datacard and then compile.

In [ ]:
# Let's open the datacard and take a look
with open("datacard_part5.txt","r") as f:
    lines = f.readlines()
    
print("".join(lines))

In [ ]:
%%bash
text2workspace.py datacard_part5.txt -m 125

The `RooMultiPdf` is a handy object for performing bias studies as all functions can be stored in a single workspace. You can then set which function is used for generating the toys with the `--setParameters pdfindex=i` option, and which function is used for fitting with `--setParameters pdfindex=i --freezeParameters pdfindex=j` options. 
* It would be a useful exercise (if time permits) to repeat the bias studies from part 4 but using the RooMultiPdf workspace. What happens when you do not freeze the index in the fitting step?

But simpler bias studies are not the only benefit of using the `RooMultiPdf`! It also allows us to apply the [discrete profiling method](https://arxiv.org/pdf/1408.6865.pdf) in our analysis. In this method, the index labelling which pdf is active (a discrete nuisance parameter) is left floating in the fit, and will be profiled by looping through all the possible index values and finding the pdf which gives the best fit. In this manner, we are able to account for the **uncertainty in the choice of the background function**. 

Note, by default, the multipdf will tell combine to add 0.5 to the NLL for each parameter in the pdf. This is known as the penalty term (or correction factor) for the discrete profiling method. You can toggle this term when building the workspace with the command `multipdf.setCorrectionFactor(0.5)`. You may need to change the value of this term to obtain an acceptable bias in your fit!

Let's run a likelihood scan using the compiled datacard with the `RooMultiPdf`:

In [ ]:
%%bash
combine -M MultiDimFit datacard_part5.root -m 125 --freezeParameters MH \
-n .scan.multidimfit --algo grid --points 20 --cminDefaultMinimizerStrategy 0 \
--saveSpecifiedIndex pdfindex_Tag0 --setParameterRanges r=0.5,2.5

The option `--cminDefaultMinimizerStrategy 0` is required to prevent HESSE being called as this cannot handle discrete nuisance parameters. HESSE is the full calculation of the second derivative matrix (Hessian) of the likelihood using finite difference methods.

The option `--saveSpecifiedIndex pdfindex_Tag0` saves the value of the index at each point in the likelihood scan. Let's have a look at how the index value changes as a function of the signal strength. You can make the following plot by running:

In [ ]:
# Open file with fits
f = ROOT.TFile("higgsCombine.scan.multidimfit.MultiDimFit.mH125.root")
t = f.Get("limit")

r, pdfindex = [], []

for ev in t:
    r.append( getattr(ev,"r") )
    pdfindex.append( getattr(ev,"pdfindex_Tag0") )

gr = ROOT.TGraph()
for i in range( len(r) ):
    gr.SetPoint( gr.GetN(), r[i], pdfindex[i] )

gr.GetXaxis().SetTitle("r")
gr.GetYaxis().SetTitle("pdfindex_Tag0")

gr.SetMarkerStyle(20)
gr.SetMarkerSize(1.5)
gr.SetLineWidth(0)

can = ROOT.TCanvas()
gr.Draw()

can.Update()
can.Draw()

By floating the discrete nuisance parameter `pdfindex_Tag0`, at each point in the likelihood scan the pdfs will be iterated over and the one which gives the max likelihood (lowest 2NLL) including the correction factor will be used. The plot above shows that the `pdfindex_Tag0=0` (exponential) is chosen for the majority of r values, but this switches to `pdfindex_Tag0=1` (Chebychev polynomial) at the lower edge of the r range. We can see the impact on the likelihood scan by fixing the background pdf to the exponential:

In [ ]:
%%bash
combine -M MultiDimFit datacard_part5.root -m 125 --freezeParameters MH,pdfindex_Tag0 \
--setParameters pdfindex_Tag0=0 -n .scan.multidimfit.fix_exp --algo grid --points 20 \
--cminDefaultMinimizerStrategy 0 --saveSpecifiedIndex pdfindex_Tag0 --setParameterRanges r=0.5,2.5

Plotting the two scans on the same axis:

In [ ]:
%%bash
plot1DScan.py higgsCombine.scan.multidimfit.MultiDimFit.mH125.root --main-label "Pdf choice floating" \
--main-color 1 --others higgsCombine.scan.multidimfit.fix_exp.MultiDimFit.mH125.root:"Pdf fixed to exponential":2 \
-o part5_scan --y-cut 35 --y-max 35

In [ ]:
# Lets open the png file and plot it here
Image(filename='part5_scan.png', width=500) 

The impact on the likelihood scan is evident at the lower edge, where the scan in which the index is floating flattens out. In this example, neither the $1\sigma$ or $2\sigma$ intervals are affected. But this is not always the case! Ultimately, this method allows us to account for the uncertainty in the choice of background function in the signal strength measurement. 

Coming back to the bias studies. Do you now understand what you are testing if you do not freeze the index in the fitting stage? In this case you are fitting the toys back with the discrete profiling method. This is the standard approach for the bias studies when we use the discrete-profiling method in an analysis.

There are a number of options which can be added to the combine command to improve the performance when using discrete nuisance parameters. These are detailed at the end of this [section](https://cms-analysis.github.io/HiggsAnalysis-CombinedLimit/part3/nonstandard/#discrete-profiling) in the combine manual.